# Reinforcement Learning Basics for our project

[Open me on Google collab](https://colab.research.google.com/github/leonardotorresaltez/routing-model-2025/blob/main/notebooks/reinforcement_learning_basic.ipynb)

This Jupyter notebook is a proof of concept (PoC) for reinforcement learning applied to our use case. It includes the basic elements: the environment, the policy, the agent (pending), and the training loop.

In this example:

- The nodes represent random (x, y) coordinates.

- The starting node is selected at random.

- The goal is to reach all remaining nodes while minimizing the total distance traveled.




## Configurations

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random

# Configuración de dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Trabajando con: {device}")

Trabajando con: cpu


## The environment

The environment is like a snapshot of a single moment:

- nodes
- current node
- nodes already visited

In [7]:
class Environment:
    def __init__(self, num_nodes):
        self.num_nodes = num_nodes
        self.reset()

    def reset(self):
        #input matrix of graph node coordinates (x,y)  num_nodes rows and 2 columns
        self.nodes = torch.rand(self.num_nodes, 2)
        #choose a random starting node
        self.current = random.randint(0, self.num_nodes - 1)
        #track visited nodes, all initially unvisited
        self.visited = torch.zeros(self.num_nodes, dtype=torch.bool)
        #mark starting node as visited
        self.visited[self.current] = True
        #track the tour
        self.tour = [self.current]
        return self.get_state()

    def get_state(self):
        return self.nodes, self.current, self.visited

    def step(self, action):
        prev = self.current
        self.current = action
        self.visited[action] = True
        self.tour.append(action)

        dist = torch.norm(self.nodes[prev] - self.nodes[action])
        reward = -dist

        done = self.visited.all()
        return self.get_state(), reward, done

## The policy ( implemented with Attention )

This is a policy learning model, which optimizes its learnable parameters during the training loop.

In this example, it is an attention-based policy. It implements an attention mechanism to determine the next node to visit.

In [8]:
class AttentionPolicy(nn.Module):
    def __init__(self, node_dim=2, embed_dim=128):
        super().__init__()
        self.node_embed = nn.Linear(node_dim, embed_dim)

        self.query = nn.Linear(embed_dim, embed_dim)
        self.key = nn.Linear(embed_dim, embed_dim)

    def forward(self, nodes, current_node_idx, visited_mask):
        """
        nodes: [N, node_dim]             node features (e.g. coordinates)
        current_node_idx: int            current position
        visited_mask: [N] (bool)         True = already visited
        """

        # Embed nodes
        h = self.node_embed(nodes)       # [N, embed_dim]

        # Query = embedding of current node
        q = self.query(h[current_node_idx])   # [embed_dim]

        # Keys = all nodes
        k = self.key(h)                       # [N, embed_dim]

        # Attention scores
        scores = torch.matmul(k, q)           # [N]

        # Mask visited nodes, the agent cannot revisit them
        scores = scores.masked_fill(visited_mask, float("-inf"))

        # Policy = probability of choosing next node
        # política estocástica
        probs = F.softmax(scores, dim=0)

        return probs
    

## Agent

The agent choose the action using the policy ( neural network algoritm )

In [9]:
class Agent:
    def __init__(self, node_dim=2, embed_dim=128, lr=1e-3):
        self.policy = AttentionPolicy(node_dim, embed_dim)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=lr)

        # Buffers for REINFORCE
        self.log_probs = []
        self.rewards = []

    def act(self, state):
        """
        state = (nodes, current_node, visited_mask)
        """
        nodes, current, visited = state

        probs = self.policy(nodes, current, visited.clone())
        dist = torch.distributions.Categorical(probs)
        action = dist.sample()

        self.log_probs.append(dist.log_prob(action))
        return action.item()

    def store_reward(self, reward):
        self.rewards.append(reward)

    def update(self):
        """
        Policy Gradient (REINFORCE)
        """
        R = sum(self.rewards)
        loss = -torch.stack(self.log_probs).sum() * R

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Clear buffers
        self.log_probs.clear()
        self.rewards.clear()

## Traingin Loop

The training loop is simple; as we can see, the model is successfully learning. It is necessary to clone the "visited" tensor to avoid a PyTorch error related to in-place operations.

In [ ]:

def train():
    env = Environment(num_nodes=10)
    agent = Agent()


    for episode in range(500):
        state = env.reset()
        done = False
        episode_reward = 0.0 

        while not done:
            action = agent.act(state)
            state, reward, done = env.step(action)
            
            agent.store_reward(reward)       
            episode_reward += reward.item()
        agent.update()



        if episode % 50 == 0:
            print(
                f"Episode {episode:4d} | "
                f"Total reward: {episode_reward:.3f}"
            )



train()

Episode 0, total reward: -4.03
Episode 50, total reward: -3.62
Episode 100, total reward: -4.57
Episode 150, total reward: -4.25
Episode 200, total reward: -2.10
Episode 250, total reward: -3.22
Episode 300, total reward: -3.10
Episode 350, total reward: -2.85
Episode 400, total reward: -4.29
Episode 450, total reward: -3.28
